In [2]:
import os, glob
import numpy as np
import pandas as pd

# PLOTTING LIBRARIES
from corner import corner
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from matplotlib.patches import Ellipse

# ASTROPY
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D, CCDData
from astropy.convolution import Gaussian2DKernel as Gauss
from astropy.convolution import convolve

# SHERPA 4.14 (installed with CIAO using Conda)
from sherpa.models.parameter import Parameter
from sherpa_contrib.profiles import *
from sherpa.astro.ui import *
from sherpa.plot import *
from sherpa.astro import xspec

# IPYWIDGETS
from IPython.display import clear_output
from ipywidgets import interact, interactive, interactive_output, Label, Layout, IntSlider, FloatSlider, HBox, VBox, Text, FloatRangeSlider, SelectMultiple, FloatLogSlider
import ipywidgets as widgets

layout = Layout(width='350px')
Float = lambda x,xm,xp,s: FloatSlider(value=x, min=xm, max=xp, step=round((xp-xm)/50,7), layout=layout, continuous_update=False, description=s)
Log = lambda x,xm,xp,s: FloatLogSlider(value=x, base=10, min=xm, max=xp, step=round((xp-xm)/50,7), layout=layout, continuous_update=False, description=s)

button = lambda string: widgets.Button(description=string,disabled=False,
                                       value=False,
                                       button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                       tooltip=string,
                                       icon='check')

def box(string,value=False,layout='65px'): 
    return widgets.Checkbox(value=value, description=string, disabled=False, indent=False, layout=Layout(width=layout))

q1, q3 = 0.15865525, 0.84134475

galaxies = glob.glob("*.fits")

In [3]:
def load_galaxy(galaxy, select_scale):
    clean()
    reset()
    global path
    path = galaxy

    file = fits.open(path)
    data = file[0].data
    shape = data.shape
    X0 = shape[0] / 2
    scale = int(shape[0])

    load_image(path)
    if select_scale != "original":
        scale = int(select_scale.split()[0].split("x")[0])
        if scale > X0*2:
            with output:
                print("Invalid scale. Original image is smaller.")
        else:
            with output:
                clear_output()
            notice2d(f"BOX({X0+0.5},{X0},{scale},{scale})")

    else:
        with output_scale:
            clear_output()
            print(f"     Original scale: {scale} x {scale}")
            
    set_coord("image")
    return X0, shape

def choose_model(select_model, X0):
    reset()
    if select_model == "Single beta":
        m = set_full_model((beta2d.b1 + const2d.bkg))
    if select_model == "Single beta + gauss":
        m = set_full_model((beta2d.b1 + gauss2d.g1 + const2d.bkg ))
    elif select_model == "Double beta":
        m = set_full_model((beta2d.b1 + beta2d.b2 + const2d.bkg))
    elif select_model == "Double beta + gauss":
        m = set_full_model((beta2d.b1 + beta2d.b2 + gauss2d.g1 + const2d.bkg))
    elif select_model == "Triple beta":
        m = set_full_model((beta2d.b1 + beta2d.b2 + beta2d.b3 + const2d.bkg))
    
    set_par(b1.xpos, val=X0, min=0, max=2*X0)
    set_par(b1.ypos, val=X0, min=0, max=2*X0)
    set_par(b1.ellip, val=0, min=0, max=0.7)
    set_par(b1.theta, val=0)
    set_par(b1.r0, val=10, min=1e-2, max=1e3)
    set_par(b1.ampl, val=20, min=1e-2, max=2e3)
    set_par(b1.alpha, val=1, min=0.1, max=10)

    if select_model in ["Double beta", "Double beta + gauss", "Triple beta"]:
        set_par(b2.xpos, val=X0, min=0, max=2*X0)
        set_par(b2.ypos, val=X0, min=0, max=2*X0)
        set_par(b2.ellip, val=0, min=0, max=0.7)
        set_par(b2.theta, val=0)
        set_par(b2.r0, val=100, min=1e-2, max=1e3)
        set_par(b2.ampl, val=5, min=1e-2, max=2e3)
        set_par(b2.alpha, val=1, min=0.1, max=10)
        
    if select_model in ["Triple beta"]:
        set_par(b3.xpos, val=X0, min=0, max=2*X0)
        set_par(b3.ypos, val=X0, min=0, max=2*X0)
        set_par(b3.ellip, val=0, min=0, max=0.7)
        set_par(b3.theta, val=0)
        set_par(b3.r0, val=1, min=1e-2, max=1e3)
        set_par(b3.ampl, val=100, min=1e-2, max=2e3)
        set_par(b3.alpha, val=1, min=0.1, max=10)
        
    if select_model in ["Single beta + gauss", "Double beta + gauss"]:
        set_par(g1.xpos, val=X0, min=X0-20, max=X0+20)
        set_par(g1.ypos, val=X0, min=X0-20, max=X0+20)
        set_par(g1.ellip, val=0, min=0, max=0.7)
        set_par(g1.theta, val=0)
        set_par(g1.fwhm, val=1, min=1e-3, max=2e1)
        set_par(g1.ampl, val=50, min=1e-3, max=1e5)     
        
    set_par(bkg.c0, val=1e-3, min=1e-6, max=5)

    #guess(b1)

def actualize(b):
    if model in ["Single beta"]:
        x0.value, y0.value, ellip.value, theta.value, r0.value, ampl.value, alpha.value, c0.value = b1.xpos.val, b1.ypos.val, b1.ellip.val, b1.theta.val, b1.r0.val, b1.ampl.val, b1.alpha.val, bkg.c0.val

    elif model in ["Single beta + gauss"]:
        x0.value, y0.value, ellip.value, theta.value, r0.value, ampl.value, alpha.value, c0.value, fwhm.value, ampl_g.value = b1.xpos.val, b1.ypos.val, b1.ellip.val, b1.theta.val, b1.r0.val, b1.ampl.val, b1.alpha.val, bkg.c0.val, g1.fwhm.val, g1.ampl.val

    elif model in ["Double beta"]:
        x0.value, y0.value, ellip.value, theta.value, r0.value, ampl.value, alpha.value, c0.value, x02.value, y02.value, ellip2.value, theta2.value, r02.value, ampl2.value, alpha2.value = b1.xpos.val, b1.ypos.val, b1.ellip.val, b1.theta.val, b1.r0.val, b1.ampl.val, b1.alpha.val, bkg.c0.val, b2.xpos.val, b2.ypos.val, b2.ellip.val, b2.theta.val, b2.r0.val, b2.ampl.val, b2.alpha.val

    elif model in ["Double beta + gauss"]:
        x0.value, y0.value, ellip.value, theta.value, r0.value, ampl.value, alpha.value, c0.value, x02.value, y02.value, ellip2.value, theta2.value, r02.value, ampl2.value, alpha2.value, fwhm.value, ampl_g.value = b1.xpos.val, b1.ypos.val, b1.ellip.val, b1.theta.val, b1.r0.val, b1.ampl.val, b1.alpha.val, bkg.c0.val, b2.xpos.val, b2.ypos.val, b2.ellip.val, b2.theta.val, b2.r0.val, b2.ampl.val, b2.alpha.val, g1.fwhm.val, g1.ampl.val

    elif model in ["Triple beta"]:
        x0.value, y0.value, ellip.value, theta.value, r0.value, ampl.value, alpha.value, c0.value, x02.value, y02.value, ellip2.value, theta2.value, r02.value, ampl2.value, alpha2.value, x03.value, y03.value, ellip3.value, theta3.value, r03.value, ampl3.value, alpha3.value = b1.xpos.val, b1.ypos.val, b1.ellip.val, b1.theta.val, b1.r0.val, b1.ampl.val, b1.alpha.val, bkg.c0.val, b2.xpos.val, b2.ypos.val, b2.ellip.val, b2.theta.val, b2.r0.val, b2.ampl.val, b2.alpha.val, b3.xpos.val, b3.ypos.val, b3.ellip.val, b3.theta.val, b3.r0.val, b3.ampl.val, b3.alpha.val

def make_plot():
    plt.close()
    fig = plt.figure(figsize=(5.5,5.5))
    get_model_prof_prefs()["linestyle"] = "-"
    get_data_prof_prefs()["xlog"] = True
    get_data_prof_prefs()["ylog"] = True
    get_delchi_prof_prefs()["xlog"] = True
    prof_fit_delchi(model=b1)
    
    ax1 = fig.add_axes([1.05, 0.1, 0.8, 0.8])
    ax1.set_title("Data")
    data = get_data().y.reshape(get_data().shape)
    X0 = data.shape[0] / 2

    if scale == "original": SCALE = X0
    else: SCALE = float(scale.split()[0].split("x")[0]) / 2

    zoom = X0 / SCALE
    X1, Y1, R1, E1, A1 = b1.xpos.val / zoom, b1.ypos.val / zoom, r0.value, ellip.value, theta.value * 180 / np.pi
    if model in ["Double beta", "Double beta + gauss", "Triple beta"]:
        X2, Y2, R2, E2, A2 = b2.xpos.val / zoom, b2.ypos.val / zoom, r02.value, ellip2.value, theta2.value * 180 / np.pi
    
    ticks = np.linspace(0, 2*SCALE // 64 * 64, 5)

    ellipse = Ellipse((X1,Y1), R1*2, R1*2 * (1 - E1), angle=A1, ec="w", fill=False, ls="--")
    if model in ["Double beta", "Double beta + gauss", "Triple beta"]:
        ellipse2 = Ellipse((X2,Y2), R2*2, R2*2 * (1 - E2), angle=A2, ec="w", fill=False, ls="--")
    
    data = data[int(X0-SCALE):int(X0+SCALE),int(X0-SCALE):int(X0+SCALE)]
    data = convolve(data, boundary = "wrap", nan_treatment="fill", fill_value = np.amin(data),
                    kernel = Gauss(x_stddev = 3))
    ax1.imshow(data+1, origin="lower", norm=LogNorm())
    ax1.plot(X1, Y1, "+r", ms=15)
    ax1.set_xlabel("x (pixels)")
    ax1.set_ylabel("y (pixels)")
    ax1.set_xticks(ticks)
    ax1.set_yticks(ticks)
    
    ax1.add_patch(ellipse)
    ax1.text(X1, Y1 + R1+1, "r0", va="bottom", ha="center", color="w")
    if model in ["Double beta", "Double beta + gauss", "Triple beta"]:
        ax1.add_patch(ellipse2)
        ax1.text(X2, Y2 + R2+1, "r02", va="bottom", ha="center", color="w")

    cmap = plt.cm.viridis
    cmaplist = [cmap(i) for i in range(cmap.N)]

    cmap = LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, 30)
        
    ax2 = fig.add_axes([0.05, -0.85, 0.8, 0.8])
    ax2.set_title("Model")
    data = get_model_image().y.reshape(get_data().shape)
    data = data[int(X0-SCALE):int(X0+SCALE),int(X0-SCALE):int(X0+SCALE)]
    ax2.imshow(data+1, origin="lower", norm=LogNorm(), cmap=cmap)
    ax2.plot(X1, Y1, "+r", ms=15)
    ax2.set_xlabel("x (pixels)")
    ax2.set_ylabel("y (pixels)")
    ax2.set_xticks(ticks)
    ax2.set_yticks(ticks)

    ax3 = fig.add_axes([1.05, -0.85, 0.8, 0.8])
    ax3.set_title("Residual")
    data = get_resid_image().y.reshape(get_data().shape)
    data = data[int(X0-SCALE):int(X0+SCALE),int(X0-SCALE):int(X0+SCALE)]
    data = convolve(data, boundary = "wrap", nan_treatment="fill", fill_value = np.amin(data),
                   kernel = Gauss(x_stddev = 3, y_stddev = 3))
    ax3.imshow(data+1, origin="lower")
    ax3.plot(X1, Y1, "+r", ms=15)
    ax3.set_xlabel("x (pixels)")
    ax3.set_ylabel("y (pixels)")
    ax3.set_xticks(ticks)
    ax3.set_yticks(ticks)

    plt.show()
        
def save_current_model(b):
    os.system("ds9 Residuals/IC4296_resid_128.fits")
    
    if scale == "original": SCALE = scale
    else: SCALE = scale.split()[0]
    with open(f"Prefits/{galaxy}_{SCALE}.txt", "w") as file:
        print(get_model(), file=file)
    
    with output:
        print("\nModel saved to", f"{galaxy}_{SCALE}.txt")

def save_current_resid(b):
    if scale == "original": SCALE = scale
    else: SCALE = scale.split()[0]
    
    file = fits.open(path)
    wcs = WCS(file[0].header)
    
    resid = get_resid_image().y.reshape(get_data().shape)
    ccd = CCDData(resid, unit="adu", wcs=wcs)
    ccd.write(f"Residuals/{galaxy}_resid_{SCALE}.fits", overwrite=True)
    
    with output:
        print("\nResidual saved to", f"{galaxy}_resid_{SCALE}.fits")
        
def fit_current_model(b):
    with output:
        clear_output()
        
        print("Making a fit. This might take time.\n")
        fit()

def texg(x, xp, xm, n):
    r = -(int(np.log10(xm)) - n)
    return "${0}_{{-{1}}}^{{+{2}}}$".format(round(x,r), round(xm,r), round(xp,r))

def run_MCMC(b):
    with output:
        covar()

        res = get_covar_results()

        set_sampler("metropolismh")
        set_sampler_opt('defaultprior', True)
        
        clear_output()
        print(f"Runing MCMC simulation with {length} iterations.\nFirst {burn} iterations will be burned.\n")
    
    stats, accept, params = get_draws(1, niter=length)

    burned = stats[:burn]
    stats, accept, params = stats[burn:], accept[burn:], params[:,burn:]

    df = pd.DataFrame(data=params.T, columns=res.parnames)
    cols = df.columns
    size = 12 * (5/len(cols))**0.7

    table = df.describe(percentiles=[q1, 0.5, q3]).T[["15.9%", "50%", "84.1%"]]
    table["Minus"] = table["50%"] - table["15.9%"]
    table["Plus"] = table["84.1%"] - table["50%"]
    table["Median"] = table["50%"]
    table = table.round(3)
    table["Param"] = table.index
    table = table[["Param", "Median", "Minus", "Plus"]]
        
    with output:
        # CORNER PLOT
        fig = plt.figure(figsize=(8,8))
        
        fig = corner(df, color="C0", fig=fig, #quantiles=[0.16, 0.5, 0.84],
                     labels=cols, #label_kwargs={"fontsize" : size-2}, 
                     labelpad=0.09,
                     show_titles=True) #, title_kwargs={"fontsize": size-2})

        # QUANTILES
        axes = np.array(fig.axes).reshape((len(cols), len(cols)))
        for i, col in enumerate(df.columns):
            ax = axes[i, i]
            ax.set_title(texg(table.Median.loc[col], table.Plus.loc[col], table.Minus.loc[col], 2), size=size)
            ax.axvline(np.quantile(df[col], q1), color="C2", ls="--", lw=2.5)
            ax.axvline(np.quantile(df[col], 0.5), color="C1", ls="--", lw=2.5)
            ax.axvline(np.quantile(df[col], q3), color="C2", ls="--", lw=2.5)

        ax1 = fig.add_axes([0.30, 0.871, 0.30, 0.1])
        ax1.set_title("Trace")
        ax1.plot(stats, c="C2")
        ax1.plot(stats[:burn], c="C1", label="burned")
        #ax1.set_xlabel("Iteration")
        ax1.set_ylabel(statistic, labelpad=2)
        ax1.set_yticklabels([])
        ax1.legend(frameon=False)
        
        axt = fig.add_axes([0.64, 0.97, 0.34, 0.001])
        axt.table(cellText=table.values, colLabels=table.columns, fontsize=size+1,
                  loc='bottom', cellLoc="right", colLoc="center").scale(1, 2.5)
        axt.axis("off")
            
        plt.show()

In [4]:
select_model = widgets.Dropdown(
    options=['Single beta', 'Single beta + gauss', 'Double beta', 'Double beta + gauss', 'Triple beta'],
    value='Single beta', 
    description='Model:', 
    disabled=False)

select_galaxy = widgets.Dropdown(
    options=galaxies,
    value="NGC4649.fits",
    description="Galaxy:",
    disabled=False)

select_scale = widgets.Dropdown(
    options=["original", "128 x 128 pixels", "256 x 256 pixels", "384 x 384 pixels", "512 x 512 pixels", "640 x 640 pixels", "768 x 768 pixels"],
    value="original",
    description="Scale:",
    disabled=False)

select_method = widgets.Dropdown(
    options=["levmar", "neldermead", "moncar"],
    value="levmar",
    description="Method:",
    disabled=False)

select_statistic = widgets.Dropdown(
    options=["cstat", "cash", "chi2", "chi2xspecvar"],
    value="cstat",
    description="Statistic:",
    disabled=False)

button_layout = Layout(width='150px')

save_button = widgets.Button(
    description="Save to txt",
    value=False,
    layout=button_layout)

save_resid_button = widgets.Button(
    description="Save residual",
    value=False,
    layout=button_layout)

length_text = widgets.Text(
    value='1000',
    placeholder='100000',
    description='Chain length:',
    disabled=False
)

burn_text = widgets.Text(
    value='100',
    placeholder='10000',
    description='Chain burn:',
    disabled=False
)

beta1 = widgets.HTML(value="<h3 style='text-align:center;margin:auto'>Beta2d.b1</h3>", description='')
beta2 = widgets.HTML(value="<h3 style='text-align:center;margin:auto'>Beta2d.b2</h3>", description='')
beta3 = widgets.HTML(value="<h3 style='text-align:center;margin:auto'>Beta2d.b3</h3>", description='')
gauss1 = widgets.HTML(value="<h3 style='text-align:center;margin:auto'>Gauss2d.g1</h3>", description='')
bkg1 = widgets.HTML(value="<h3 style='text-align:center;margin:auto'>Const2d.bkg</h3>", description='')

def h_blank(h=16):
    blank = HBox([])
    blank.layout = Layout(height=f"{h}px")
    return blank

def v_blank(h=50):
    blank = VBox([])
    blank.layout = Layout(width=f"{h}px")
    return blank

output = widgets.Output()

output2 = widgets.Output()

output3 = widgets.Output()

output_scale = widgets.Output()

output.layout = Layout(width='750px')

output_scale.layout = Layout(width='265px')

fit_button = widgets.Button(
    description="Fit",
    value=False,
    button_style='success',
    layout=button_layout)

MCMC_button = widgets.Button(
    description="Run MCMC",
    value=False,
    layout=button_layout)

# ON-CLICK BUTTONS
save_button.on_click(save_current_model)

save_resid_button.on_click(save_current_resid)

fit_button.on_click(fit_current_model)
fit_button.on_click(actualize)

MCMC_button.on_click(run_MCMC)

def scale_to_original(change):
    select_scale.value="original"
    
def model_to_single(change):
    select_model.value="Single beta"

def method_to_levmar(change):
    select_method.value="levmar"
    
def statistic_to_cstat(change):
    select_statistic.value="cstat"
    
select_galaxy.observe(scale_to_original)

select_galaxy.observe(model_to_single)

select_galaxy.observe(method_to_levmar)

select_galaxy.observe(statistic_to_cstat)

TraitError: Invalid selection: value not found

In [ ]:
def main(select_galaxy, select_model, select_scale):
    with output:
        clear_output()
    
    X0, shape = load_galaxy(select_galaxy, select_scale)
    
    global galaxy
    galaxy = select_galaxy
    
    global scale
    scale = select_scale
    
    global model
    model = select_model

    choose_model(select_model, X0)

    # CREATE SLIDERS & BUTTONS
    global x0, y0, ellip, theta, r0, ampl, alpha, c0
    x0 = Float(b1.xpos.val, b1.xpos.min, b1.xpos.max, "X0")
    y0 = Float(b1.ypos.val, b1.ypos.min, b1.ypos.max, "Y0")
    x0_f = box("freeze"); y0_f = box("freeze")

    ellip = Float(b1.ellip.val, b1.ellip.min, b1.ellip.max, "ellip")
    theta = Float(b1.theta.val, b1.theta.min, b1.theta.max, "theta")
    ellip_f = box("freeze", value=True); theta_f = box("freeze", value=True); 

    r0 = Log(b1.r0.val, np.log10(b1.r0.min), np.log10(b1.r0.max), "r0")
    ampl = Log(b1.ampl.val, np.log10(b1.ampl.min), np.log10(b1.ampl.max), "ampl")
    alpha = Log(b1.alpha.val, np.log10(b1.alpha.min), np.log10(b1.alpha.max), "alpha")
    r0_f = box("freeze"); ampl_f = box("freeze"); alpha_f = box("freeze")

    if select_model in ["Double beta", "Double beta + gauss", "Triple beta"]:
        global x02, y02, ellip2, theta2, r02, ampl2, alpha2
        x02 = Float(b2.xpos.val, b2.xpos.min, b2.xpos.max, "X0_2")
        y02 = Float(b2.ypos.val, b2.ypos.min, b2.ypos.max, "Y0_2")
        x02_f = box("freeze"); y02_f = box("freeze")
        x02_l = box("link", value=True, layout='45px'); y02_l = box("link", value=True, layout='45px')

        ellip2 = Float(b2.ellip.val, b2.ellip.min, b2.ellip.max, "ellip_2")
        theta2 = Float(b2.theta.val, b2.theta.min, b2.theta.max, "theta_2")
        ellip2_f = box("freeze", value=True); ellip2_l = box("link", value=True, layout='45px')
        theta2_f = box("freeze", value=True); theta2_l = box("link", value=True, layout='45px')

        r02 = Log(b2.r0.val, np.log10(b2.r0.min), np.log10(b2.r0.max), "r0_2")
        ampl2 = Log(b2.ampl.val, np.log10(b2.ampl.min), np.log10(b2.ampl.max), "ampl_2")
        alpha2 = Log(b2.alpha.val, np.log10(b2.alpha.min), np.log10(b2.alpha.max), "alpha_2")
        r02_f = box("freeze"); ampl2_f = box("freeze"); alpha2_f = box("freeze"); alpha2_l = box("link", value=True, layout='45px')

    if select_model in ["Triple beta"]:
        global x03, y03, ellip3, theta3, r03, ampl3, alpha3
        x03 = Float(b3.xpos.val, b3.xpos.min, b3.xpos.max, "X0_3")
        y03 = Float(b3.ypos.val, b3.ypos.min, b3.ypos.max, "Y0_3")
        x03_f = box("freeze"); y03_f = box("freeze")
        x03_l = box("link", value=True, layout='45px'); y03_l = box("link", value=True, layout='45px')

        ellip3 = Float(b3.ellip.val, b3.ellip.min, b3.ellip.max, "ellip_3")
        theta3 = Float(b3.theta.val, b3.theta.min, b3.theta.max, "theta_3")
        ellip3_f = box("freeze", value=True); ellip3_l = box("link", value=True, layout='45px')
        theta3_f = box("freeze", value=True); theta3_l = box("link", value=True, layout='45px')
        
        r03 = Log(b3.r0.val, np.log10(b3.r0.min), np.log10(b3.r0.max), "r0_3")
        ampl3 = Log(b3.ampl.val, np.log10(b3.ampl.min), np.log10(b3.ampl.max), "ampl_3")
        alpha3 = Log(b3.alpha.val, np.log10(b3.alpha.min), np.log10(b3.alpha.max), "alpha_3")
        r03_f = box("freeze"); ampl3_f = box("freeze"); alpha3_f = box("freeze"); alpha3_l = box("link", value=True, layout='45px')
    
    if select_model in ["Single beta + gauss", "Double beta + gauss"]:
        global x0g, y0g, fwhm, ampl_g
        x0g = Float(g1.xpos.val, g1.xpos.min, g1.xpos.max, "X0_g")
        y0g = Float(g1.ypos.val, g1.ypos.min, g1.ypos.max, "Y0_g")
        x0g_f = box("freeze"); y0g_f = box("freeze")
        x0g_l = box("link", value=True, layout='45px'); y0g_l = box("link", value=True, layout='45px')

        fwhm = Log(g1.fwhm.val, np.log10(g1.fwhm.min), np.log10(g1.fwhm.max), "fwhm")
        ampl_g = Log(g1.ampl.val, np.log10(g1.ampl.min), np.log10(g1.ampl.max), "ampl_g")
        fwhm_f = box("freeze"); ampl_g_f = box("freeze")

    c0 = Log(bkg.c0.val, np.log10(bkg.c0.min), np.log10(bkg.c0.max), "c0")
    c0_f = box("freeze")
    

    # CHOOSING PARAMS AND SLIDERS
    params = {"length_text" : length_text, "burn_text" : burn_text, "select_statistic" : select_statistic, "select_method" : select_method,
              "x0" : x0, "x0_f" : x0_f, "y0" : y0, "y0_f" : y0_f, "c0" : c0, "c0_f" : c0_f,
              "ellip" : ellip, "ellip_f" : ellip_f, "theta" : theta, "theta_f" : theta_f,
              "r0" : r0, "r0_f" : r0_f, "ampl" : ampl, "ampl_f" : ampl_f, "alpha" : alpha, "alpha_f" : alpha_f}
    
    buttons = VBox([h_blank(16),
                    HBox([select_method, length_text]),
                    HBox([select_statistic, burn_text]),
                    h_blank(16),
                    HBox([fit_button, MCMC_button, save_button, save_resid_button])]) #, plot_button])])
    
    single_beta = VBox([beta1,
                        HBox([x0, x0_f]), 
                        HBox([y0, y0_f]),
                        HBox([ellip, ellip_f]),
                        HBox([theta, theta_f]),
                        HBox([r0, r0_f]),
                        HBox([ampl, ampl_f]), 
                        HBox([alpha, alpha_f])])
    
    background = VBox([bkg1,
                       HBox([c0, c0_f])])
                       
    
    if select_model in ["Double beta", "Double beta + gauss", "Triple beta"]:
        double_beta = VBox([beta2,
                            HBox([x02, x02_f, x02_l]), 
                            HBox([y02, y02_f, y02_l]),
                            HBox([ellip2, ellip2_f, ellip2_l]),
                            HBox([theta2, theta2_f, theta2_l]),
                            HBox([r02, r02_f]), 
                            HBox([ampl2, ampl2_f]), 
                            HBox([alpha2, alpha2_f, alpha2_l])])
    
    if select_model in ["Triple beta"]:
        triple_beta = VBox([beta3,
                            HBox([x03, x03_f, x03_l]), 
                            HBox([y03, y03_f, y03_l]),
                            HBox([ellip3, ellip3_f, ellip3_l]),
                            HBox([theta3, theta3_f, theta3_l]),
                            HBox([r03, r03_f]), 
                            HBox([ampl3, ampl3_f]), 
                            HBox([alpha3, alpha3_f, alpha3_l])])

    if select_model in ["Single beta + gauss", "Double beta + gauss"]:
        gauss = VBox([gauss1,
                      HBox([x0g, x0g_f, x0g_l]), 
                      HBox([y0g, y0g_f, y0g_l]),
                      HBox([fwhm, fwhm_f]),
                      HBox([ampl_g, ampl_g_f])])
    
    if select_model in ["Single beta"]:
        sliders = HBox([single_beta, background])
        
    elif select_model in ["Single beta + gauss"]:
        sliders = HBox([single_beta, gauss, background])
        
        params = {**params, "x0g" : x0g, "x0g_f" : x0g_f, "x0g_l" : x0g_l, "y0g" : y0g, "y0g_f" : y0g_f,  "y0g_l" : y0g_l,
                  "fwhm" : fwhm, "fwhm_f" : fwhm_f, "ampl_g" : ampl_g, "ampl_g_f" : ampl_g_f}
            
    elif select_model in ["Double beta"]:
        sliders = HBox([single_beta, double_beta, background])
        
        params = {**params, "x02" : x02, "x02_f" : x02_f, "x02_l" : x02_l, "y02" : y02, "y02_f" : y02_f,  "y02_l" : y02_l,
                  "ellip2" : ellip2, "ellip2_f" : ellip2_f, "ellip2_l" : ellip2_l, "theta2" : theta2, "theta2_f" : theta2_f, "theta2_l" : theta2_l,
                  "r02" : r02, "r02_f" : r02_f, "ampl2" : ampl2, "ampl2_f" : ampl2_f, "alpha2" : alpha2, "alpha2_f" : alpha2_f, "alpha2_l" : alpha2_l}

    elif select_model in ["Double beta + gauss"]:
        sliders = HBox([single_beta, double_beta, gauss, background])
        
        params = {**params, "x02" : x02, "x02_f" : x02_f, "x02_l" : x02_l, "y02" : y02, "y02_f" : y02_f,  "y02_l" : y02_l,
                  "ellip2" : ellip2, "ellip2_f" : ellip2_f, "ellip2_l" : ellip2_l, "theta2" : theta2, "theta2_f" : theta2_f, "theta2_l" : theta2_l,
                  "r02" : r02, "r02_f" : r02_f, "ampl2" : ampl2, "ampl2_f" : ampl2_f, "alpha2" : alpha2, "alpha2_f" : alpha2_f, "alpha2_l" : alpha2_l,
                  "x0g" : x0g, "x0g_f" : x0g_f, "x0g_l" : x0g_l, "y0g" : y0g, "y0g_f" : y0g_f,  "y0g_l" : y0g_l,
                  "fwhm" : fwhm, "fwhm_f" : fwhm_f, "ampl_g" : ampl_g, "ampl_g_f" : ampl_g_f}
        
    elif select_model in ["Triple beta"]:
        sliders = HBox([single_beta, double_beta, triple_beta, background])

        params = {**params, "x02" : x02, "x02_f" : x02_f, "x02_l" : x02_l, "y02" : y02, "y02_f" : y02_f,  "y02_l" : y02_l,
                  "ellip2" : ellip2, "ellip2_f" : ellip2_f, "ellip2_l" : ellip2_l, "theta2" : theta2, "theta2_f" : theta2_f, "theta2_l" : theta2_l,
                  "r02" : r02, "r02_f" : r02_f, "ampl2" : ampl2, "ampl2_f" : ampl2_f, "alpha2" : alpha2, "alpha2_f" : alpha2_f, "alpha2_l" : alpha2_l,
                  "x03" : x03, "x03_f" : x03_f, "x03_l" : x03_l, "y03" : y03, "y03_f" : y03_f,  "y03_l" : y03_l,
                  "ellip3" : ellip3, "ellip3_f" : ellip3_f, "ellip3_l" : ellip3_l, "theta3" : theta3, "theta3_f" : theta3_f, "theta3_l" : theta3_l,
                  "r03" : r03, "r03_f" : r03_f, "ampl3" : ampl3, "ampl3_f" : ampl3_f, "alpha3" : alpha3, "alpha3_f" : alpha3_f, "alpha3_l" : alpha3_l}
    
    out = interactive_output(change_params, params)
            
    display(VBox([sliders, VBox([HBox([VBox([buttons, output]), out]), output2])]))

def change_params(select_statistic="cstat", select_method="levmar", length_text=1000, burn_text=100,
                  x0=0, x0_f=False, y0=0, y0_f=False, c0=0, c0_f=False,
                  ellip=0, ellip_f=False, theta=0, theta_f=False,
                  r0=0, r0_f=False, ampl=0, ampl_f=False, alpha=0, alpha_f=False,
                  x02=0, x02_f=False, x02_l=False, y02=0, y02_f=False, y02_l=False,
                  ellip2=0, ellip2_f=False, ellip2_l=False, theta2=0, theta2_f=False, theta2_l=False,
                  r02=0, r02_f=False, ampl2=0, ampl2_f=False, alpha2=0, alpha2_f=False, alpha2_l=False,
                  x03=0, x03_f=False, x03_l=False, y03=0, y03_f=False, y03_l=False,
                  ellip3=0, ellip3_f=False, ellip3_l=False, theta3=0, theta3_f=False, theta3_l=False,
                  r03=0, r03_f=False, ampl3=0, ampl3_f=False, alpha3=0, alpha3_f=False, alpha3_l=False,
                  x0g=0, x0g_f=False, x0g_l=False, y0g=0, y0g_f=False, y0g_l=False,
                  fwhm=0, fwhm_f=False, ampl_g=0, ampl_g_f=False):
         
    set_stat(select_statistic)
    set_method(select_method)
    
    global statistic
    statistic = select_statistic
        
    global length, burn
    length, burn = int(length_text), int(burn_text)

    # SINGLE BETA
    if x0_f: freeze(b1.xpos)
    else: 
        thaw(b1.xpos)
        set_par(b1.xpos, val=x0)
    
    if y0_f: freeze(b1.ypos)
    else: 
        thaw(b1.ypos)
        set_par(b1.ypos, val=y0)
    
    if ellip_f: freeze(b1.ellip)
    else: 
        thaw(b1.ellip)
        set_par(b1.ellip, val=ellip)
        
    if theta_f: freeze(b1.theta)
    else: 
        thaw(b1.theta)
        set_par(b1.theta, val=theta)
    
    if r0_f: freeze(b1.r0)
    else: 
        thaw(b1.r0)
        set_par(b1.r0, val=r0)
    
    if ampl_f: freeze(b1.ampl)
    else: 
        thaw(b1.ampl)
        set_par(b1.ampl, val=ampl)
    
    if alpha_f: freeze(b1.alpha)
    else: 
        thaw(b1.alpha)
        set_par(b1.alpha, val=alpha)

    # DOUBLE BETA
    try:
        if r02_f: freeze(b2.r0)
        else:
            thaw(b2.r0)
            set_par(b2.r0, val=r02)

        if ampl2_f: freeze(b2.ampl)
        else:
            thaw(b2.ampl)
            set_par(b2.ampl, val=ampl2)

        if alpha2_l: b2.alpha = b1.alpha
        else:
            unlink(b2.alpha)
            if alpha2_f: freeze(b2.alpha)
            else:
                thaw(b2.alpha)
                set_par(b2.alpha, val=alpha2)

        if ellip2_l: b2.ellip = b1.ellip
        else:
            unlink(b2.ellip)
            if ellip2_f: freeze(b2.ellip)
            else: 
                thaw(b2.ellip)
                set_par(b2.ellip, val=ellip2)

        if theta2_l: b2.theta = b1.theta
        else:
            unlink(b2.theta)
            if theta2_f: freeze(b2.theta)
            else: 
                thaw(b2.theta)
                set_par(b2.theta, val=theta2)

        if x02_l: b2.xpos = b1.xpos
        else:
            unlink(b2.xpos)
            if x02_f: freeze(b2.xpos)
            else: 
                thaw(b2.xpos)
                set_par(b2.xpos, val=x02)

        if y02_l: b2.ypos = b1.ypos
        else:
            unlink(b2.ypos)
            if y02_f: freeze(b2.ypos)
            else: 
                thaw(b2.ypos)
                set_par(b2.ypos, val=y02)
    except: pass

    # TRIPLE BETA
    try:
        if r03_f: freeze(b3.r0)
        else:
            thaw(b3.r0)
            set_par(b3.r0, val=r03)

        if ampl3_f: freeze(b3.ampl)
        else:
            thaw(b3.ampl)
            set_par(b3.ampl, val=ampl3)

        if alpha3_l: b3.alpha = b1.alpha
        else:
            unlink(b3.alpha)
            if alpha3_f: freeze(b3.alpha)
            else:
                thaw(b3.alpha)
                set_par(b3.alpha, val=alpha3)
            
        if ellip3_l: b3.ellip = b1.ellip
        else:
            unlink(b3.ellip)
            if ellip3_f: freeze(b3.ellip)
            else: 
                thaw(b3.ellip)
                set_par(b3.ellip, val=ellip3)

        if theta3_l: b3.theta = b1.theta
        else:
            unlink(b3.theta)
            if theta3_f: freeze(b3.theta)
            else: 
                thaw(b3.theta)
                set_par(b3.theta, val=theta3)
            
        if x03_l: b3.xpos = b1.xpos
        else:
            unlink(b3.xpos)
            if x03_f: freeze(b3.xpos)
            else: 
                thaw(b3.xpos)
                set_par(b3.xpos, val=x03)

        if y03_l: b3.ypos = b1.ypos
        else:
            unlink(b3.ypos)
            if y03_f: freeze(b3.ypos)
            else: 
                thaw(b3.ypos)
                set_par(b3.ypos, val=y03)
    except: pass

    # GAUSS
    try:
        if fwhm_f: freeze(g1.fwhm)
        else:
            thaw(g1.fwhm)
            set_par(g1.fwhm, val=fwhm)

        if ampl_g_f: freeze(g1.ampl)
        else:
            thaw(g1.ampl)
            set_par(g1.ampl, val=ampl_g)

        if x0g_l: g1.xpos = b1.xpos
        else:
            unlink(g1.xpos)
            if x0g_f: freeze(g1.xpos)
            else: 
                thaw(g1.xpos)
                set_par(g1.xpos, val=x0g)

        if y0g_l: g1.ypos = b1.ypos
        else:
            unlink(g1.ypos)
            if y0g_f: freeze(g1.ypos)
            else:
                thaw(g1.ypos)
                set_par(g1.ypos, val=y0g)
    except: pass

    # BACKGROUND
    if c0_f: freeze(bkg.c0)
    else: 
        thaw(bkg.c0)
        set_par(bkg.c0, val=c0)

    make_plot()


out2 = interactive_output(main, {"select_galaxy" : select_galaxy, 
                                 "select_model" : select_model, 
                                 "select_scale" : select_scale})

tvl = HBox([v_blank(0),
            VBox([VBox([HBox([select_galaxy, output_scale]),
            HBox([select_model, select_scale])]), out2])])

tvl2 = widgets.Text(description="nichts")

tab = widgets.Tab([tvl, tvl2])
tab.set_title(0, 'Beta modelling')
tab.set_title(1, 'Cavity detection')

display(tab)